# Initial Exploratory Data Analysis

Using the filtered data set, investigate the need for any additional cleaning, manipulation, imputation, or other tasks keeping in mind the desire to format the data for scikit-learn clustering algorithms.

## Preliminaries

Load the libraries

In [123]:
import pandas as pd
import numpy as np
import datetime
from pandas_profiling import ProfileReport
from pandas import CategoricalDtype


In [124]:
# configure pandas to display all rows and columns without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [125]:
inFileName = "1..SAMFilteredEntityData.csv"

Read the data, setting null values to empty strings with the `na_filter=False` option.

In [126]:
df = pd.read_csv(inFileName, na_filter=False)

Look at the number of rows and columns.

In [127]:
df.shape

(9860, 13)

Look at the data types `read_csv` used as defaults.

In [128]:
df.dtypes

CAGE_CODE                           object
INITIAL_REGISTRATION_DATE            int64
PHYSICAL_ADDRESS_ZIP_POSTAL_CODE     int64
ENTITY_CONGRESSIONAL_DISTRICT        int64
BUSINESS_START_DATE                  int64
FISCAL_YEAR_END_CLOSE_DATE           int64
ENTITY_STRUCTURE                    object
STATE_OF_INCORPORATION              object
BUSINESS_TYPE_STRING                object
NAICS_CODE_STRING                   object
PSC_CODE_STRING                     object
CREDIT_CARD_USAGE                   object
SBA_BUSINESS_TYPES_STRING           object
dtype: object

In [129]:
df.head()

,CAGE_CODE,INITIAL_REGISTRATION_DATE,PHYSICAL_ADDRESS_ZIP_POSTAL_CODE,ENTITY_CONGRESSIONAL_DISTRICT,BUSINESS_START_DATE,FISCAL_YEAR_END_CLOSE_DATE,ENTITY_STRUCTURE,STATE_OF_INCORPORATION,BUSINESS_TYPE_STRING,NAICS_CODE_STRING,PSC_CODE_STRING,CREDIT_CARD_USAGE,SBA_BUSINESS_TYPES_STRING
0,3GUY7,20030725,54806,7,19781012,331,2L,WI,2X,221320Y~236220Y~238220Y~238910Y,,N,XX
1,1TCA0,20020401,52806,2,20000605,1231,2L,IA,27~2X~8E~8W~A2~XS,425120Y~484110Y~484121Y~484122Y~484220Y~484230...,R602~V112~V129,Y,A620220819
2,1CK93,19971210,3570,2,19450101,1231,2L,NH,2X~XS,115310E~237310Y~321113Y~321912Y~321920Y~321999...,4010~5130~5305~5306~5310~5315~5510~5520~5530~6...,Y,XX
3,4Z748,20040816,5038,0,19570901,1231,2L,VT,2X~8W~A2~MF~XS,332119Y~332510Y~332618Y~332722Y~332999Y~333517...,,N,XX
4,6V169,20010615,1089,1,19450915,1231,2J,MA,27~2X~MF~XS,331529Y,9530~9540,Y,XX


## Observations

- PHYSICAL_ADDRESS_ZIP_POSTAL_CODE lost the leading zeroes and needs coersion to a string
- INITIAL_REGISTRATION_DATE and BUSINESS_START_DATE fields need coersion to datetime
- FISCAL_YEAR_END_CLOSE_DATE requires some thought
- BUSINESS_TYPE_STRING, NAICS_CODE_STRING, and PSC_CODE_STRING require binarization
- SBA_BUSINESS_TYPES_STRING requires investigation
- All others need coersion to a categorical type


## Dealing with Loss of Leading Zeroes in the Zip Code

In [140]:
df = pd.read_csv(inFileName, na_filter=False, dtype={'PHYSICAL_ADDRESS_ZIP_POSTAL_CODE' : str}, engine='python')
df.dtypes

CAGE_CODE                           object
INITIAL_REGISTRATION_DATE            int64
PHYSICAL_ADDRESS_ZIP_POSTAL_CODE    object
ENTITY_CONGRESSIONAL_DISTRICT        int64
BUSINESS_START_DATE                  int64
FISCAL_YEAR_END_CLOSE_DATE           int64
ENTITY_STRUCTURE                    object
STATE_OF_INCORPORATION              object
BUSINESS_TYPE_STRING                object
NAICS_CODE_STRING                   object
PSC_CODE_STRING                     object
CREDIT_CARD_USAGE                   object
SBA_BUSINESS_TYPES_STRING           object
dtype: object

In [141]:
df.head()

,CAGE_CODE,INITIAL_REGISTRATION_DATE,PHYSICAL_ADDRESS_ZIP_POSTAL_CODE,ENTITY_CONGRESSIONAL_DISTRICT,BUSINESS_START_DATE,FISCAL_YEAR_END_CLOSE_DATE,ENTITY_STRUCTURE,STATE_OF_INCORPORATION,BUSINESS_TYPE_STRING,NAICS_CODE_STRING,PSC_CODE_STRING,CREDIT_CARD_USAGE,SBA_BUSINESS_TYPES_STRING
0,3GUY7,20030725,54806,7,19781012,331,2L,WI,2X,221320Y~236220Y~238220Y~238910Y,,N,XX
1,1TCA0,20020401,52806,2,20000605,1231,2L,IA,27~2X~8E~8W~A2~XS,425120Y~484110Y~484121Y~484122Y~484220Y~484230...,R602~V112~V129,Y,A620220819
2,1CK93,19971210,03570,2,19450101,1231,2L,NH,2X~XS,115310E~237310Y~321113Y~321912Y~321920Y~321999...,4010~5130~5305~5306~5310~5315~5510~5520~5530~6...,Y,XX
3,4Z748,20040816,05038,0,19570901,1231,2L,VT,2X~8W~A2~MF~XS,332119Y~332510Y~332618Y~332722Y~332999Y~333517...,,N,XX
4,6V169,20010615,01089,1,19450915,1231,2J,MA,27~2X~MF~XS,331529Y,9530~9540,Y,XX


## Dealing with the Date Fields







In [142]:
df['BUSINESS_START_DATE'] = pd.to_datetime(df.BUSINESS_START_DATE, format='%Y%m%d', errors='coerce')
df['INITIAL_REGISTRATION_DATE'] = pd.to_datetime(df.BUSINESS_START_DATE, format='%Y%m%d', errors='coerce')

In [143]:
df.dtypes

CAGE_CODE                                   object
INITIAL_REGISTRATION_DATE           datetime64[ns]
PHYSICAL_ADDRESS_ZIP_POSTAL_CODE            object
ENTITY_CONGRESSIONAL_DISTRICT                int64
BUSINESS_START_DATE                 datetime64[ns]
FISCAL_YEAR_END_CLOSE_DATE                   int64
ENTITY_STRUCTURE                            object
STATE_OF_INCORPORATION                      object
BUSINESS_TYPE_STRING                        object
NAICS_CODE_STRING                           object
PSC_CODE_STRING                             object
CREDIT_CARD_USAGE                           object
SBA_BUSINESS_TYPES_STRING                   object
dtype: object

## Dealing with the Categorical Variables

The following fields should be considered categorical data:

- CAGE_CODE
- PHYSICAL_ADDRESS_ZIP_POSTAL_CODE
- ENTITY_CONGRESSIONAL_DISTRICT 
- BUSINESS_START_DATE 
- FISCAL_YEAR_END_CLOSE_DATE  
- ENTITY_STRUCTURE    
- STATE_OF_INCORPORATION           
- CREDIT_CARD_USAGE           


In [145]:
df['CAGE_CODE'] = df['CAGE_CODE'].astype('category')
df['PHYSICAL_ADDRESS_ZIP_POSTAL_CODE'] = df['PHYSICAL_ADDRESS_ZIP_POSTAL_CODE'].astype('category')
df['ENTITY_CONGRESSIONAL_DISTRICT'] = df['ENTITY_CONGRESSIONAL_DISTRICT'].astype('category')
df['FISCAL_YEAR_END_CLOSE_DATE'] = df['FISCAL_YEAR_END_CLOSE_DATE'].astype('category')
df['ENTITY_STRUCTURE'] = df['ENTITY_STRUCTURE'].astype('category')
df['STATE_OF_INCORPORATION'] = df['STATE_OF_INCORPORATION'].astype('category')
df['CREDIT_CARD_USAGE'] = df['CREDIT_CARD_USAGE'].astype('category')
       
df.dtypes

CAGE_CODE                                 category
INITIAL_REGISTRATION_DATE           datetime64[ns]
PHYSICAL_ADDRESS_ZIP_POSTAL_CODE          category
ENTITY_CONGRESSIONAL_DISTRICT             category
BUSINESS_START_DATE                 datetime64[ns]
FISCAL_YEAR_END_CLOSE_DATE                category
ENTITY_STRUCTURE                          category
STATE_OF_INCORPORATION                    category
BUSINESS_TYPE_STRING                        object
NAICS_CODE_STRING                           object
PSC_CODE_STRING                             object
CREDIT_CARD_USAGE                         category
SBA_BUSINESS_TYPES_STRING                   object
dtype: object

In [146]:
df.head()

,CAGE_CODE,INITIAL_REGISTRATION_DATE,PHYSICAL_ADDRESS_ZIP_POSTAL_CODE,ENTITY_CONGRESSIONAL_DISTRICT,BUSINESS_START_DATE,FISCAL_YEAR_END_CLOSE_DATE,ENTITY_STRUCTURE,STATE_OF_INCORPORATION,BUSINESS_TYPE_STRING,NAICS_CODE_STRING,PSC_CODE_STRING,CREDIT_CARD_USAGE,SBA_BUSINESS_TYPES_STRING
0,3GUY7,1978-10-12,54806,7,1978-10-12,331,2L,WI,2X,221320Y~236220Y~238220Y~238910Y,,N,XX
1,1TCA0,2000-06-05,52806,2,2000-06-05,1231,2L,IA,27~2X~8E~8W~A2~XS,425120Y~484110Y~484121Y~484122Y~484220Y~484230...,R602~V112~V129,Y,A620220819
2,1CK93,1945-01-01,03570,2,1945-01-01,1231,2L,NH,2X~XS,115310E~237310Y~321113Y~321912Y~321920Y~321999...,4010~5130~5305~5306~5310~5315~5510~5520~5530~6...,Y,XX
3,4Z748,1957-09-01,05038,0,1957-09-01,1231,2L,VT,2X~8W~A2~MF~XS,332119Y~332510Y~332618Y~332722Y~332999Y~333517...,,N,XX
4,6V169,1945-09-15,01089,1,1945-09-15,1231,2J,MA,27~2X~MF~XS,331529Y,9530~9540,Y,XX


## Next Steps

Add ProfileReport here and see where that leads.

Further analysis captured in seperate notebooks require the same preprocessing steps. Need to figure out the best method for optimizing that.

The *STRING columns require binarization. When and where should that be done?